In [1]:
import tensorflow as tf
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from keras.backend.tensorflow_backend import set_session
config = tf.ConfigProto()
config.gpu_options.per_process_gpu_memory_fraction = 0.3
set_session(tf.Session(config=config))

%matplotlib inline

Using TensorFlow backend.


In [2]:
features = ['Malignancy', 'Diameter', 'Lobulation', 'Spiculation']

In [3]:
%run 'CNNBlocks.ipynb'
%run 'modelArchitecture.ipynb'
%run 'batchGenerator.ipynb'

In [4]:
model = compileModel((1,64,64,64),5e-4)

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:5: UserWarning: Update your `Conv3D` call to the Keras 2 API: `Conv3D(8, (3, 3, 3), data_format="channels_first", padding="same", kernel_regularizer=<keras.reg...)`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:12: UserWarning: Update your `MaxPooling3D` call to the Keras 2 API: `MaxPooling3D(data_format="channels_first")`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:6: UserWarning: Update your `AveragePooling3D` call to the Keras 2 API: `AveragePooling3D(data_format="channels_first")`
/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:7: UserWarning: The `merge` function is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
/opt/anaconda/lib/python3.5/site-packages/keras/legacy/layers.py:456: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras

Compiling model...


In [ ]:
# model_64.load_weights('/home/katya/LungCancer/Katya/CNN_v1/model_v1_weights_temp64.h5')

nbEpochs = 15

modelPath = '/home/katya/LungCancer/Katya/CNN_v2/model_and_weights/LUNA_model_v2.h5'
model, lossHist = trainModel(model, modelPath, batchSize=50, nbEpoch = nbEpochs, stepsPerEpoch = 150)

model.save_weights('/home/katya/LungCancer/Katya/CNN_v2/model_and_weights/LUNA_model_v2_weights.h5')

Loading positive patches
The number of samples for true category constitutes 3570
Loading negative patches
The number of samples for random category constitutes 5328


/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:57: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=150, callbacks=[<keras.ca..., validation_steps=20, validation_data=<generator..., epochs=1, initial_epoch=0)`


Epoch 1/1
150/150 [==============================] - 698s - loss: 0.6376 - Malignancy_loss: 0.1435 - Diameter_loss: 0.0708 - Lobulation_loss: 0.0631 - Spiculation_loss: 0.0971 - val_loss: 0.5663 - val_Malignancy_loss: 0.0883 - val_Diameter_loss: 0.0708 - val_Lobulation_loss: 0.0671 - val_Spiculation_loss: 0.0774

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:57: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=150, callbacks=[<keras.ca..., validation_steps=20, validation_data=<generator..., epochs=2, initial_epoch=1)`


150/150 [==============================] - 699s - loss: 0.4918 - Malignancy_loss: 0.0893 - Diameter_loss: 0.0400 - Lobulation_loss: 0.0455 - Spiculation_loss: 0.0545 - val_loss: 0.4971 - val_Malignancy_loss: 0.0924 - val_Diameter_loss: 0.0415 - val_Lobulation_loss: 0.0469 - val_Spiculation_loss: 0.0541

...

150/150 [==============================] - 699s - loss: 0.4218 - Malignancy_loss: 0.0702 - Diameter_loss: 0.0260 - Lobulation_loss: 0.0323 - Spiculation_loss: 0.0342 - val_loss: 0.4500 - val_Malignancy_loss: 0.0746 - val_Diameter_loss: 0.0341 - val_Lobulation_loss: 0.0389 - val_Spiculation_loss: 0.0437

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:57: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=150, callbacks=[<keras.ca..., validation_steps=20, validation_data=<generator..., epochs=8, initial_epoch=7)`


150/150 [==============================] - 699s - loss: 0.4169 - Malignancy_loss: 0.0696 - Diameter_loss: 0.0233 - Lobulation_loss: 0.0314 - Spiculation_loss: 0.0343 - val_loss: 0.4475 - val_Malignancy_loss: 0.0813 - val_Diameter_loss: 0.0304 - val_Lobulation_loss: 0.0399 - val_Spiculation_loss: 0.0377

/opt/anaconda/lib/python3.5/site-packages/ipykernel/__main__.py:57: UserWarning: Update your `fit_generator` call to the Keras 2 API: `fit_generator(<generator..., steps_per_epoch=150, callbacks=[<keras.ca..., validation_steps=20, validation_data=<generator..., epochs=9, initial_epoch=8)`


 32/150 [=====>........................] - ETA: 516s - loss: 0.4095 - Malignancy_loss: 0.0667 - Diameter_loss: 0.0225 - Lobulation_loss: 0.0296 - Spiculation_loss: 0.0328

In [ ]:
plt.figure(figsize=[10,7])
plt.grid(True, ls='--', lw=0.5, alpha=0.5, dash_capstyle = 'round', c='gray')
plt.xlabel('Epoch #', fontsize=15)
plt.ylabel('Loss', fontsize=15)

labels=list(lossHist.keys())

for i,key in enumerate(lossHist):
    plt.plot([x for x in range(nbEpochs)], lossHist[key], 'o-', label=labels[i])
    plt.legend()
    
plt.savefig('/home/katya/LungCancer/Katya/CNN_v2/figures/' + str(version) + '.png')

In [ ]:
version = 7

In [ ]:
for feature in features:
    plt.figure(figsize=[10,8])
    plt.grid(True, ls='--', lw=0.5, alpha=0.5, dash_capstyle = 'round', c='gray')

    plt.plot([x for x in range(nbEpochs)], lossHist[feature+'_loss'], label=feature+'_loss')
    plt.plot([x for x in range(nbEpochs)], lossHist['val_'+feature+'_loss'], label='val_'+feature+'_loss')
    plt.title(feature)
    plt.legend()
    plt.savefig('/home/katya/LungCancer/Katya/CNN_v2/figures/'+str(version)+'.'+feature+'.png')